In [12]:
from anytree import Node, RenderTree

class CKYParser:
    def __init__(self, grammar):
        self.grammar = grammar

    def parse(self, sentence):
        words = sentence.split()
        n = len(words)
        parse_table = [[set() for _ in range(n)] for _ in range(n)]
        tree_table = [[[] for _ in range(n)] for _ in range(n)]

        # Initialize the table with the grammar rules for each word
        for j in range(n):
            for lhs, rhs in self.grammar.items():
                if words[j] in rhs:
                    parse_table[j][j].add(lhs)
                    tree_table[j][j].append(Node(lhs, word=words[j]))

        # Fill the parse table and tree table
        for length in range(2, n + 1):
            for start in range(n - length + 1):
                end = start + length - 1
                for mid in range(start, end):
                    for lhs, productions in self.grammar.items():
                        for production in productions:
                            if len(production) == 2:
                                left, right = production
                                if left in parse_table[start][mid] and right in parse_table[mid + 1][end]:
                                    parent = Node(lhs)
                                    for left_tree in tree_table[start][mid]:
                                        for right_tree in tree_table[mid + 1][end]:
                                            new_parent = Node(lhs)
                                            left_tree.parent = new_parent
                                            right_tree.parent = new_parent
                                            tree_table[start][end].append(new_parent)

                                    parse_table[start][end].add(lhs)

        return parse_table, tree_table

    def print_parse_table(self, parse_table):
        for i in range(len(parse_table)):
            for j in range(len(parse_table)):
                if parse_table[i][j]:
                    print(f"Parse Table[{i}][{j}]: {parse_table[i][j]}")

    def print_parse_tree(self, tree_table):
        if tree_table[0][len(tree_table) - 1]:
            print("Parse Tree:")
            root = tree_table[0][len(tree_table) - 1][0]
            for pre, _, node in RenderTree(root):
                if hasattr(node, 'word'):
                    print(f"{pre}{node.name} (word: {node.word})")
                else:
                    print(f"{pre}{node.name}")
        else:
            print("No valid parse tree found.")

# Cebuano grammar in CNF
grammar = {
    'S': {('VP',)},  # Sentence can be a VP
    'VP': {('Verb', 'NP'), ('Verb', 'PP'), ('Verb',)},  # VP can have a noun phrase or prepositional phrase
    'NP': {('Det', 'Noun'), ('Det', 'Adj', 'Noun'), ('Pronoun',)},  # Noun phrase structures
    'PP': {('Preposition', 'NP')},  # Prepositional phrase structure
    'Det': {'ang', 'usa ka'},  # Determiners
    'Noun': {'kuting', 'iro', 'tao', 'teleskopyo'},  # Nouns
    'Pronoun': {'ako', 'ikaw', 'siya'},  # Pronouns
    'Verb': {'nag kuting', 'nag tan-aw', 'nag kaon', 'nag mag', 'mag kaon', 'nag', 'mag'},  # Various verb forms
    'Adj': {'maayong', 'gamay'},  # Adjectives
    'Preposition': {'sa', 'uban sa'}  # Prepositions
}

# Instantiate the CKY parser
cky_parser = CKYParser(grammar)

# Parse a Cebuano sentence
sentence = "nag kuting nag tan-aw sa iro"  # Example sentence
parse_table, tree_table = cky_parser.parse(sentence)

# Print the parse table
cky_parser.print_parse_table(parse_table)

# Print the parse tree
cky_parser.print_parse_tree(tree_table)

Parse Table[0][0]: {'Verb'}
Parse Table[1][1]: {'Noun'}
Parse Table[2][2]: {'Verb'}
Parse Table[4][4]: {'Preposition'}
Parse Table[5][5]: {'Noun'}
No valid parse tree found.
